### Phase 2 – Variable Selection and Modeling Building

1. Data Cleaning

- For this phase use only the binned training data set.
- For any variable with missing values,change the data toinclude a missing category instead of a missing value for the categorical variable. (df.fillna())
- Check each variable for separation concerns. Document in the report and adjust any variables with complete or quasi-separation concerns.

2. Model Building

- Use backward selection to do the variable selection – the Bank currently uses 𝛼=0.002 and p-values to perform backward, but is open to another technique and/or significance level if documented in your report.

- Report the final variables from this model ranked by p-value.(HINT:Even if you choose to not use p-values tos elect your variables,you should still rank all final variables by their p-value in this report.)

In [7]:
# import needed libraries
import pandas as pd
import numpy as np
import scipy as sp
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact
from scipy.stats import spearmanr
from scipy.stats.contingency import association
from statsmodels.gam.api import GLMGam, BSplines
from statsmodels.genmod.families import Binomial
import statsmodels.formula.api as smf
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.generalized_linear_model import GLM


### Get the Data

In [8]:
train_df = pd.read_csv("insurance_t_bin.csv")

train_df.head(10)

,DDA,CASHBK,DIRDEP,NSF,SAV,ATM,CD,IRA,LOC,INV,...,INVBAL_BIN,ILSBAL_BIN,MMBAL_BIN,MTGBAL_BIN,CCBAL_BIN,INCOME_BIN,LORES_BIN,HMVAL_BIN,AGE_BIN,CRSCORE_BIN
0,1,0,1,0,0,1,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,01 <= 0.9,01 <= 50,02 > 5,01 <= 98,02 > 50,01 <= 675
1,0,0,0,0,0,0,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,01 <= 0.9,01 <= 50,02 > 5,01 <= 98,02 > 50,01 <= 675
2,1,0,1,0,0,0,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,02 > 0.9,01 <= 50,01 <= 5,02 <= 122,02 > 50,01 <= 675
3,1,0,0,1,1,1,0,0,0,NaN,...,00 Miss,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,00 Miss,01 <= 50,01 <= 5,02 <= 122,01 <= 50,01 <= 675
4,1,0,0,1,1,1,1,0,0,NaN,...,00 Miss,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,00 Miss,01 <= 50,02 > 5,01 <= 98,02 > 50,02 > 675
5,0,0,0,0,1,1,0,0,0,NaN,...,00 Miss,01 <= 9690.09,02 > 1031.1401,01 <= 100000,00 Miss,01 <= 50,02 > 5,01 <= 98,02 > 50,02 > 675
6,1,0,0,1,1,0,0,0,0,NaN,...,00 Miss,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,00 Miss,01 <= 50,02 > 5,02 <= 122,02 > 50,01 <= 675
7,0,0,0,0,1,0,0,0,1,NaN,...,00 Miss,01 <= 9690.09,01 <= 1031.1401,02 > 100000,00 Miss,01 <= 50,01 <= 5,02 <= 122,01 <= 50,01 <= 675
8,1,0,1,0,0,0,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,01 <= 0.9,02 > 50,02 > 5,03 <= 142,01 <= 50,01 <= 675
9,0,0,0,0,1,0,0,1,0,NaN,...,00 Miss,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,00 Miss,01 <= 50,01 <= 5,01 <= 98,02 > 50,02 > 675


#### First Up: How many (proportionally) are buying the product?

In [9]:
pd.crosstab(index = train_df['INS'], columns = "prop")/pd.crosstab(index = train_df['INS'], columns = "prop").sum()

col_0,prop
INS,
0,0.656504
1,0.343496


### CHeck for Missing Values, and fill with "M"

In [10]:
# get a count of missing values by column:
train_df.isna().mean()

DDA            0.000000
CASHBK         0.000000
DIRDEP         0.000000
NSF            0.000000
SAV            0.000000
ATM            0.000000
CD             0.000000
IRA            0.000000
LOC            0.000000
INV            0.126545
ILS            0.000000
MM             0.000000
MMCRED         0.000000
MTG            0.000000
CC             0.126545
CCPURC         0.126545
SDB            0.000000
HMOWN          0.172219
MOVED          0.000000
INAREA         0.000000
INS            0.000000
BRANCH         0.000000
RES            0.000000
ACCTAGE_BIN    0.000000
DDABAL_BIN     0.000000
DEP_BIN        0.000000
DEPAMT_BIN     0.000000
CHECKS_BIN     0.000000
NSFAMT_BIN     0.000000
PHONE_BIN      0.000000
TELLER_BIN     0.000000
SAVBAL_BIN     0.000000
ATMAMT_BIN     0.000000
POS_BIN        0.000000
POSAMT_BIN     0.000000
CDBAL_BIN      0.000000
IRABAL_BIN     0.000000
LOCBAL_BIN     0.000000
INVBAL_BIN     0.000000
ILSBAL_BIN     0.000000
MMBAL_BIN      0.000000
MTGBAL_BIN     0

In [11]:
# step one - fill all Missing values with "M"

train_df = train_df.fillna(value="M")

train_df.head()


,DDA,CASHBK,DIRDEP,NSF,SAV,ATM,CD,IRA,LOC,INV,...,INVBAL_BIN,ILSBAL_BIN,MMBAL_BIN,MTGBAL_BIN,CCBAL_BIN,INCOME_BIN,LORES_BIN,HMVAL_BIN,AGE_BIN,CRSCORE_BIN
0,1,0,1,0,0,1,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,01 <= 0.9,01 <= 50,02 > 5,01 <= 98,02 > 50,01 <= 675
1,0,0,0,0,0,0,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,01 <= 0.9,01 <= 50,02 > 5,01 <= 98,02 > 50,01 <= 675
2,1,0,1,0,0,0,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,02 > 0.9,01 <= 50,01 <= 5,02 <= 122,02 > 50,01 <= 675
3,1,0,0,1,1,1,0,0,0,M,...,00 Miss,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,00 Miss,01 <= 50,01 <= 5,02 <= 122,01 <= 50,01 <= 675
4,1,0,0,1,1,1,1,0,0,M,...,00 Miss,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,00 Miss,01 <= 50,02 > 5,01 <= 98,02 > 50,02 > 675


In [12]:
# check that the fillna solved the issue. It worked!

null_pct = train_df.isna().mean()

hi_null_pct = null_pct[null_pct > 0]

hi_null_pct

Series([], dtype: float64)

### Sort the Variables by Variable Type (binary, nominal, ordinal, continuous)

In [13]:
def get_variable_types(df):
    binary = []
    ordinal = []
    nominal = []
    continuous = []

    for col in df.columns:
        # print(df[col].value_counts())
        # print(df[col].nunique())
        if df[col].nunique() == 2:
            binary.append(col)
        elif df[col].nunique() >=3 and df[col].nunique() <=10:
            ordinal.append(col)
        elif df[col].nunique() >=10 and df[col].nunique() <=40:
            nominal.append(col)
        else:
            continuous.append(col)
    # print(binary)
    # print(ordinal)
    # print(nominal)
    # print(continuous)

    return binary, ordinal, nominal, continuous


In [14]:
binary, ordinal, nominal, continuous = get_variable_types(train_df)

binary, ordinal, nominal, continuous

print(f"There are {len(binary)} binary variables")
print(f"There are {len(ordinal)} ordinal variables")
print(f"There are {len(nominal)} nominal variables")
print(f"There are {len(continuous)} continuous variables")


There are 21 binary variables
There are 26 ordinal variables
There are 1 nominal variables
There are 0 continuous variables


In [15]:
print(binary)

['DDA', 'DIRDEP', 'NSF', 'SAV', 'ATM', 'CD', 'IRA', 'LOC', 'ILS', 'MM', 'MTG', 'SDB', 'MOVED', 'INAREA', 'INS', 'NSFAMT_BIN', 'IRABAL_BIN', 'LOCBAL_BIN', 'ILSBAL_BIN', 'MMBAL_BIN', 'MTGBAL_BIN']


### Check for Linear Separation Issues with Categorical Variables : One Quadrant = 0 in the Contingency Table
- if this is the case, collapse the categories, using df.replace({})

In [16]:
# Check for quasi-complete linear separation. We can do this with a Crosstab of the Categorical variable and the Categorical Target

for col in ordinal:
    cont_table = pd.crosstab(columns=train_df[col], index=train_df['INS'])
    if (cont_table == 0).any().any():
        print(col)
        print(cont_table)


CASHBK
CASHBK     0    1  2
INS                 
0       5473  102  2
1       2891   27  0
MMCRED
MMCRED     0    1   2  3  5
INS                        
0       5409  130  33  4  1
1       2713  153  47  5  0


In [17]:
# collapse the categories with linear separation
train_df['CASHBK'] = train_df['CASHBK'].replace({1:"1+", 2:"1+"})
train_df['MMCRED'] = train_df['MMCRED'].replace({3:"3+",5:"3+"})

In [18]:
#check with a cont table:
cashbk_cont = pd.crosstab(columns=train_df['CASHBK'], index=train_df['INS'])
cashbk_cont

CASHBK,0,1+
INS,,
0,5473,104
1,2891,27


In [19]:
#check with a cont table:
mmcred_cont = pd.crosstab(columns=train_df['MMCRED'], index=train_df['INS'])
mmcred_cont

MMCRED,0,1,2,3+
INS,,,,
0,5409,130,33,5
1,2713,153,47,5


### Build a LR Model - Using Backward Selection

In [20]:
pip install mlxtend statsmodels

Note: you may need to restart the kernel to use updated packages.


In [23]:
target_df = train_df
target_df = train_df.drop(columns='INS')
target_df.columns

Index(['DDA', 'CASHBK', 'DIRDEP', 'NSF', 'SAV', 'ATM', 'CD', 'IRA', 'LOC',
       'INV', 'ILS', 'MM', 'MMCRED', 'MTG', 'CC', 'CCPURC', 'SDB', 'HMOWN',
       'MOVED', 'INAREA', 'BRANCH', 'RES', 'ACCTAGE_BIN', 'DDABAL_BIN',
       'DEP_BIN', 'DEPAMT_BIN', 'CHECKS_BIN', 'NSFAMT_BIN', 'PHONE_BIN',
       'TELLER_BIN', 'SAVBAL_BIN', 'ATMAMT_BIN', 'POS_BIN', 'POSAMT_BIN',
       'CDBAL_BIN', 'IRABAL_BIN', 'LOCBAL_BIN', 'INVBAL_BIN', 'ILSBAL_BIN',
       'MMBAL_BIN', 'MTGBAL_BIN', 'CCBAL_BIN', 'INCOME_BIN', 'LORES_BIN',
       'HMVAL_BIN', 'AGE_BIN', 'CRSCORE_BIN'],
      dtype='object')

In [46]:
# Apply the unique function to each column in the DataFrame
unique_values = train_df.apply(lambda col: col.unique())

# Print the unique values for each column
for col in unique_values.index:
    print(f"Unique values in {col}: {unique_values[col]}")


Unique values in DDA: [1 0]
Unique values in CASHBK: [0 '1+']
Unique values in DIRDEP: [1 0]
Unique values in NSF: [0 1]
Unique values in SAV: [0 1]
Unique values in ATM: [1 0]
Unique values in CD: [0 1]
Unique values in IRA: [0 1]
Unique values in LOC: [0 1]
Unique values in INV: [0.0 'M' 1.0]
Unique values in ILS: [0 1]
Unique values in MM: [0 1]
Unique values in MMCRED: [0 1 2 '3+']
Unique values in MTG: [0 1]
Unique values in CC: [1.0 'M' 0.0]
Unique values in CCPURC: [1.0 0.0 'M' 2.0 3.0 4.0]
Unique values in SDB: [0 1]
Unique values in HMOWN: [1.0 0.0 'M']
Unique values in MOVED: [0 1]
Unique values in INAREA: [1 0]
Unique values in INS: [0 1]
Unique values in BRANCH: ['B2' 'B3' 'B7' 'B14' 'B15' 'B19' 'B1' 'B18' 'B17' 'B9' 'B5' 'B8' 'B4'
 'B6' 'B16' 'B10' 'B12' 'B11' 'B13']
Unique values in RES: ['R' 'S' 'U']
Unique values in ACCTAGE_BIN: ['01 <= 19.6' '00 Miss' '02 > 19.6']
Unique values in DDABAL_BIN: ['06 <= 2188.02' '01 <= 0.1' '05 <= 1248.47' '03 <= 304.95' '08 > 6126.24'
 '

In [60]:
br_values = train_df['BRANCH'].unique()
br_values.sort()
br_values

array(['B1', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17',
       'B18', 'B19', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'],
      dtype=object)

In [48]:
#create a factorized dataframe, called target_df
target_df = train_df.drop(columns=['INS'])

In [49]:
target_df.head()

,DDA,CASHBK,DIRDEP,NSF,SAV,ATM,CD,IRA,LOC,INV,...,INVBAL_BIN,ILSBAL_BIN,MMBAL_BIN,MTGBAL_BIN,CCBAL_BIN,INCOME_BIN,LORES_BIN,HMVAL_BIN,AGE_BIN,CRSCORE_BIN
0,1,0,1,0,0,1,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,01 <= 0.9,01 <= 50,02 > 5,01 <= 98,02 > 50,01 <= 675
1,0,0,0,0,0,0,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,01 <= 0.9,01 <= 50,02 > 5,01 <= 98,02 > 50,01 <= 675
2,1,0,1,0,0,0,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,02 > 0.9,01 <= 50,01 <= 5,02 <= 122,02 > 50,01 <= 675
3,1,0,0,1,1,1,0,0,0,M,...,00 Miss,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,00 Miss,01 <= 50,01 <= 5,02 <= 122,01 <= 50,01 <= 675
4,1,0,0,1,1,1,1,0,0,M,...,00 Miss,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,00 Miss,01 <= 50,02 > 5,01 <= 98,02 > 50,02 > 675


In [61]:
dummy_cols = ['CASHBK','INV','MMCRED','CC','CCPURC','HMOWN','BRANCH','RES','ACCTAGE_BIN','DDABAL_BIN','DEP_BIN','DEPAMT_BIN',
              'CHECKS_BIN', 'NSFAMT_BIN','PHONE_BIN','TELLER_BIN','SAVBAL_BIN','ATMAMT_BIN','POS_BIN','POSAMT_BIN','CDBAL_BIN',
              'IRABAL_BIN', 'LOCBAL_BIN','INVBAL_BIN', 'ILSBAL_BIN','MMBAL_BIN','MTGBAL_BIN','CCBAL_BIN', 'INCOME_BIN', 'LORES_BIN',
              'HMVAL_BIN', 'AGE_BIN', 'CRSCORE_BIN']
target_df = pd.get_dummies(train_df, columns=dummy_cols, drop_first=True)

In [70]:
target_df.columns

Index(['DDA', 'DIRDEP', 'NSF', 'SAV', 'ATM', 'CD', 'IRA', 'LOC', 'ILS', 'MM',
       ...
       'LORES_BIN_01 <= 5', 'LORES_BIN_02 > 5', 'HMVAL_BIN_01 <= 98',
       'HMVAL_BIN_02 <= 122', 'HMVAL_BIN_03 <= 142', 'HMVAL_BIN_04 > 142',
       'AGE_BIN_01 <= 50', 'AGE_BIN_02 > 50', 'CRSCORE_BIN_01 <= 675',
       'CRSCORE_BIN_02 > 675'],
      dtype='object', length=109)

In [64]:
target_df.head()

,DDA,DIRDEP,NSF,SAV,ATM,CD,IRA,LOC,ILS,MM,...,LORES_BIN_01 <= 5,LORES_BIN_02 > 5,HMVAL_BIN_01 <= 98,HMVAL_BIN_02 <= 122,HMVAL_BIN_03 <= 142,HMVAL_BIN_04 > 142,AGE_BIN_01 <= 50,AGE_BIN_02 > 50,CRSCORE_BIN_01 <= 675,CRSCORE_BIN_02 > 675
0,1,1,0,0,1,0,0,0,0,0,...,False,True,True,False,False,False,False,True,True,False
1,0,0,0,0,0,0,0,0,0,0,...,False,True,True,False,False,False,False,True,True,False
2,1,1,0,0,0,0,0,0,0,0,...,True,False,False,True,False,False,False,True,True,False
3,1,0,1,1,1,0,0,0,0,0,...,True,False,False,True,False,False,True,False,True,False
4,1,0,1,1,1,1,0,0,0,0,...,False,True,True,False,False,False,False,True,False,True


In [65]:
target_df['CASHBK_1+']

0       False
1       False
2       False
3       False
4       False
        ...  
8490    False
8491    False
8492    False
8493    False
8494    False
Name: CASHBK_1+, Length: 8495, dtype: bool

In [74]:
column_list = target_df.columns
print(column_list)

Index(['DDA', 'DIRDEP', 'NSF', 'SAV', 'ATM', 'CD', 'IRA', 'LOC', 'ILS', 'MM',
       ...
       'LORES_BIN_01 <= 5', 'LORES_BIN_02 > 5', 'HMVAL_BIN_01 <= 98',
       'HMVAL_BIN_02 <= 122', 'HMVAL_BIN_03 <= 142', 'HMVAL_BIN_04 > 142',
       'AGE_BIN_01 <= 50', 'AGE_BIN_02 > 50', 'CRSCORE_BIN_01 <= 675',
       'CRSCORE_BIN_02 > 675'],
      dtype='object', length=109)


In [67]:
# get dummies for non-binary vars
target_df = target_df.astype(int)
target_df.head()

,DDA,DIRDEP,NSF,SAV,ATM,CD,IRA,LOC,ILS,MM,...,LORES_BIN_01 <= 5,LORES_BIN_02 > 5,HMVAL_BIN_01 <= 98,HMVAL_BIN_02 <= 122,HMVAL_BIN_03 <= 142,HMVAL_BIN_04 > 142,AGE_BIN_01 <= 50,AGE_BIN_02 > 50,CRSCORE_BIN_01 <= 675,CRSCORE_BIN_02 > 675
0,1,1,0,0,1,0,0,0,0,0,...,0,1,1,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,1,1,0
2,1,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,1,0
3,1,0,1,1,1,0,0,0,0,0,...,1,0,0,1,0,0,1,0,1,0
4,1,0,1,1,1,1,0,0,0,0,...,0,1,1,0,0,0,0,1,0,1


In [68]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import statsmodels.api as sm

# Assuming `X` contains the predictor variables and `y` is the target variable
X = target_df  # Replace with actual column names
y = train_df["INS"]  # Your binary target variable (e.g., 0 or 1)

# Initialize logistic regression model
log_reg = LogisticRegression()

# Stepwise forward feature selection
sfs = SFS(log_reg,
          k_features="best",  # You can specify a number here (e.g., 3 for top 3 features)
          forward=True,
          floating=False,
          scoring="accuracy",  # Can use other metrics as well
          cv=5)

# Fit the feature selector to the data
sfs = sfs.fit(X, y)

# Get the selected feature indices
selected_features = list(sfs.k_feature_idx_)

# Use the selected features in the logistic regression model for p-values
X_selected = X.iloc[:, selected_features]
X_selected = sm.add_constant(X_selected)  # Add constant for intercept

# Fit logistic regression using statsmodels to get p-values
logit_model = sm.Logit(y, X_selected)
result = logit_model.fit()

# Display the summary, including p-values
print(result.summary())


/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is de

         Current function value: 0.000000
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                    INS   No. Observations:                 8495
Model:                          Logit   Df Residuals:                     8493
Method:                           MLE   Df Model:                            1
Date:                Fri, 06 Sep 2024   Pseudo R-squ.:                   1.000
Time:                        12:32:41   Log-Likelihood:            -2.4614e-07
converged:                      False   LL-Null:                       -5465.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -27.2460    1.1e+04     -0.002      0.998   -2.17e+04    2.16e+04
INS           50.4759   1.12e+04      0.004      0.996    -2.2e+0

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/opt/anaconda3/lib/p

In [75]:
logit_model = sm.Logit(train_df['INS'], target_df)
result = logit_model.fit()

#singular matrix - this is from the same null columns across two variables. Will need to remove one of these

# full model, highest p-value, 47 reduced models, with the categories . Full model but without the categories in variable 1,

#Full v reduced, 47 times. THen rank order the p-values - look at the highest one, if higher than cutoff then 
# remove all levels of that category

# Then run the model again, with that model reduced 

# ideas: r wrapper in python, or surrender, or remove the seperated cols

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/discrete/discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/opt/anaconda3/lib/python3.11/site-packages/statsmodels/discrete/discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix

In [ ]:
# Get p-values
p_values = result.pvalues

# Set threshold for p-value (e.g., 0.05)
significant_features = p_values[p_values < 0.002].index

print(f"Selected significant features: {significant_features}")


In [ ]:
# forward selection for the interactions